In [1]:
import os
import spacy
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer



In [7]:
# Cargar el modelo de lenguaje en ingles de SpaCy
nlp = spacy.load('en_core_web_sm')

#es_core_news_sm
# Función de preprocesamiento avanzado
def advanced_preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space and not token.is_stop]
    return tokens


# Entrenamiento

In [8]:
# Directorio donde están los archivos
directory = "ArchivosWiki"

# Preparar los datos
sentences = []
contexts = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r", encoding='utf-8') as file:
            content = file.read()
        
        # Dividir el contenido en fragmentos más pequeños
        # Puedes dividir por líneas, párrafos o un número fijo de caracteres
        max_chunk_size = 100000  # Tamaño máximo de cada fragmento
        content_chunks = [content[i:i+max_chunk_size] for i in range(0, len(content), max_chunk_size)]
        
        for chunk in content_chunks:
            # Preprocesar el fragmento
            tokens = advanced_preprocess(chunk)
            sentences.append(tokens)
            
            # Dividir el fragmento en oraciones utilizando SpaCy
            doc = nlp(chunk)
            paragraphs = [sent.text.strip() for sent in doc.sents]
            contexts.extend(paragraphs)

# Entrenar el modelo Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec_wiki.model")

# Cargar el modelo Word2Vec
model = Word2Vec.load("word2vec_wiki.model")

# Crear un DataFrame con los contextos
df = pd.DataFrame({'text': contexts})

# Generar una matriz TF-IDF para ponderar los embeddings
vectorizer = TfidfVectorizer(tokenizer=lambda x: advanced_preprocess(x), lowercase=True)
tfidf_matrix = vectorizer.fit_transform(df['text'])
tfidf_feature_names = vectorizer.get_feature_names_out()

# Crear un diccionario palabra: idf
idf_dict = dict(zip(tfidf_feature_names, vectorizer.idf_))

# Función para obtener el embedding promedio ponderado por TF-IDF de una oración
def get_weighted_sentence_embedding(sentence, model, idf_dict):
    tokens = advanced_preprocess(sentence)
    word_embeddings = []
    weights = []
    for word in tokens:
        if word in model.wv.key_to_index and word in idf_dict:
            word_embeddings.append(model.wv[word])
            weights.append(idf_dict[word])
    if not word_embeddings:
        return np.zeros(model.vector_size)
    word_embeddings = np.array(word_embeddings)
    weights = np.array(weights).reshape(-1,1)
    weighted_average = np.sum(word_embeddings * weights, axis=0) / np.sum(weights)
    return weighted_average

# Generar embeddings para cada contexto
df['embeddings'] = df['text'].apply(lambda x: get_weighted_sentence_embedding(x, model, idf_dict))

# Función para encontrar los contextos más relevantes
def find_most_relevant_contexts(question, df, model, idf_dict, top_n=3):
    question_embedding = get_weighted_sentence_embedding(question, model, idf_dict)
    context_embeddings = np.vstack(df['embeddings'].values)
    similarities = cosine_similarity([question_embedding], context_embeddings)[0]
    df['similarity'] = similarities
    top_indices = df['similarity'].argsort()[-top_n:][::-1]
    most_relevant_contexts = df.iloc[top_indices]['text'].tolist()
    return most_relevant_contexts

# Ejemplo de uso
question = "Who painted the Mona Lisa?"
most_relevant_contexts = find_most_relevant_contexts(question, df, model, idf_dict, top_n=3)

# Imprimir los contextos más relevantes
print("Pregunta:", question)
print("Contextos más relevantes:")
for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")


/home/velasco/Documentos/Proyectos/MachineLearning/Tesis/SistemaQyA/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pregunta: Who painted the Mona Lisa?
Contextos más relevantes:
1. The Mona Lisa wears green in her portrait, as does the bride in the Arnolfini portrait by Jan van Eyck.
2. Biographer Bob Colacello provides some details on Andy's "piss paintings":

Warhol's 1982 portrait of Basquiat, Jean-Michel Basquiat, is a silkscreen over an oxidized copper "piss painting".
3. Saroyan also painted.


# GUARDADO DE INFORMACION

In [9]:
import pickle

# Guardar el DataFrame 'df' con los embeddings
df.to_pickle('df_with_embeddings.pkl')

# Guardar el diccionario 'idf_dict'
with open('idf_dict.pkl', 'wb') as f:
    pickle.dump(idf_dict, f)


In [10]:
# Guardar los embeddings como un archivo numpy
embeddings_array = np.vstack(df['embeddings'].values)
np.save('embeddings.npy', embeddings_array)

# Guardar los textos
#df['text'].to_csv('texts.csv', index=False)


# LECTURA DE LOS ARCHIVOS GENERADOS

In [1]:
import pickle
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from openai import OpenAI
import os
from dotenv import load_dotenv


# Cargar variables de entorno desde un archivo .env
load_dotenv()

# Leer la API key desde las variables de entorno

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Cargar el modelo de lenguaje en ingles de SpaCy
nlp = spacy.load('en_core_web_sm')

#es_core_news_sm
# Función de preprocesamiento avanzado
def advanced_preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space and not token.is_stop]
    return tokens


# Cargar el modelo de Word2Vec
model = Word2Vec.load("ModelosEmbeddings/word2vec_wiki.model")

# Cargar el DataFrame con los textos y sus embeddings
df = pd.read_pickle('ModelosEmbeddings/df_with_embeddings.pkl')

# Cargar el diccionario 'idf_dict'
with open('ModelosEmbeddings/idf_dict.pkl', 'rb') as f:
    idf_dict = pickle.load(f)

# Cargar los embeddings como un archivo numpy
embeddings_array = np.load('ModelosEmbeddings/embeddings.npy')

# Actualizar los embeddings en el DataFrame (opcional)
df['embeddings'] = list(embeddings_array)

# Función para calcular el embedding de una pregunta
def get_weighted_sentence_embedding(sentence, model, idf_dict):
    tokens = advanced_preprocess(sentence)
    word_embeddings = []
    weights = []
    for word in tokens:
        if word in model.wv.key_to_index and word in idf_dict:
            word_embeddings.append(model.wv[word])
            weights.append(idf_dict[word])
    if not word_embeddings:
        return np.zeros(model.vector_size)
    word_embeddings = np.array(word_embeddings)
    weights = np.array(weights).reshape(-1, 1)
    weighted_average = np.sum(word_embeddings * weights, axis=0) / np.sum(weights)
    return weighted_average

# Función para encontrar los contextos más relevantes
def find_most_relevant_contexts(question, df, model, idf_dict, top_n=3):
    question_embedding = get_weighted_sentence_embedding(question, model, idf_dict)
    context_embeddings = np.vstack(df['embeddings'].values)
    similarities = cosine_similarity([question_embedding], context_embeddings)[0]
    df['similarity'] = similarities
    top_indices = df['similarity'].argsort()[-top_n:][::-1]
    most_relevant_contexts = df.iloc[top_indices]['text'].tolist()
    return most_relevant_contexts

def answer_question_word2vec(
    df,
    question,
    word2vec_model,
    idf_dict,
    top_n=3,
    gpt_model="gpt-3.5-turbo",
    max_tokens=150,
    debug=False,
    stop_sequence=None
):
    """
    Answer a question using the most relevant contexts obtained from Word2Vec.
    
    Parameters:
        df (DataFrame): The dataframe containing text and embeddings.
        question (str): The user's question.
        word2vec_model: The trained Word2Vec model.
        idf_dict (dict): Dictionary containing IDF values.
        top_n (int): Number of top relevant contexts to retrieve.
        gpt_model (str): The GPT model to use (default: "gpt-3.5-turbo").
        max_tokens (int): Maximum tokens for GPT response.
        debug (bool): If True, print debugging information.
        stop_sequence: The stop sequence for GPT response.

    Returns:
        str: The GPT-generated response based on Word2Vec contexts.
    """
    # Obtener los contextos más relevantes usando Word2Vec
    relevant_contexts = find_most_relevant_contexts(question, df, word2vec_model, idf_dict, top_n=top_n)

    # Combinar los contextos en un solo texto
    context = "\n".join(relevant_contexts)

    if debug:
        print("Contextos relevantes obtenidos de Word2Vec:\n")
        print(context)
        print("\n---\n")

    try:
        # Crear una consulta a GPT con los contextos obtenidos
        response = client.chat.completions.create(
            model=gpt_model,
            messages=[
                {
                    "role": "system",
                    "content": "Responde la pregunta basándote en el contexto proporcionado. Si no puedes responder basándote en el contexto, di 'Escribe AGENTE para más información.'"
                },
                {
                    "role": "system",
                    "content": f"Contexto: {context}"
                },
                {
                    "role": "user",
                    "content": f"Pregunta: {question}"
                }
            ],
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
        )
        return response.choices[0].message.content

    except Exception as e:
        print(f"Error al generar la respuesta: {e}")
        return ""


# PREGUNTAS FACTUALES

In [22]:
# Lista de preguntas
questions = [
    "Who painted the Mona Lisa?",
    "What is the capital of France?",
    "When did World War II begin?",
    "Who was the first man to walk on the Moon?",
    "What is the longest river in the world?",
    "Which country has the largest population?",
    "Who wrote 'One Hundred Years of Solitude'?",
    "What chemical element has the symbol 'O'?",
    "In what year was America discovered?",
    "What is the highest mountain in the world?",
    "Who is the author of the theory of relativity?",
    "What is the largest ocean in the world?",
    "Which country won the 2018 FIFA World Cup?",
    "Which planet is known as the Red Planet?",
    "Who was the first president of the United States?",
    "How many colors are in a rainbow?",
    "On which continent is Egypt located?",
    "What language is spoken in Brazil?",
    "What currency is used in Japan?",
    "What language is spoken in Japan?"
]


# Iterar sobre las preguntas y obtener respuestas
for i, question in enumerate(questions, 1):
    print(f"\nPregunta {i}: {question}")
    
    # Llamar al método para generar la respuesta
    response = answer_question_word2vec(
        df=df,
        question=question,
        word2vec_model=model,
        idf_dict=idf_dict,
        top_n=20,  # Obtener los contextos más relevantes
        gpt_model="gpt-3.5-turbo",
        max_tokens=150,
        debug=False  # Desactiva el modo de depuración para respuestas limpias
    )
    
    # Imprimir la respuesta generada
    print(f"Respuesta: {response}")



Pregunta 1: Who painted the Mona Lisa?
Respuesta: Leonardo da Vinci painted the Mona Lisa.

Pregunta 2: What is the capital of France?
Respuesta: La capital de Francia es París.

Pregunta 3: When did World War II begin?
Respuesta: La Segunda Guerra Mundial comenzó en 1939.

Pregunta 4: Who was the first man to walk on the Moon?
Respuesta: Escribe AGENTE para más información.

Pregunta 5: What is the longest river in the world?
Respuesta: La respuesta basada en el contexto proporcionado es: El río Amazonas es el río más largo del mundo.

Pregunta 6: Which country has the largest population?
Respuesta: Escribe AGENTE para más información.

Pregunta 7: Who wrote 'One Hundred Years of Solitude'?
Respuesta: Escribe AGENTE para más información.

Pregunta 8: What chemical element has the symbol 'O'?
Respuesta: Escribe AGENTE para más información.

Pregunta 9: In what year was America discovered?
Respuesta: Vespucci explored South America between 1497 and 1502, and was the first European to s

# PREGUNTAS CONTEXTUALES

In [23]:
# Lista de preguntas
questions = [
    "What does 'Python' mean in programming?",
    "What does 'Python' mean in zoology?",
    "What is Java in computer science?",
    "What is Java in geography?",
    "What is the connection between the Sun and photosynthesis?",
    "How does global warming affect sea levels?",
    "What does the melting of polar ice caps imply?",
    "What is the impact of climate change on agriculture?",
    "What are the benefits of artificial intelligence in medicine?",
    "What does 'bank' mean in the financial context?",
    "What does 'bank' mean in the context of a park?",
    "How is electricity related to magnetism?",
    "What is quantum computing and how is it used?",
    "How is cryptocurrency defined?",
    "What role does water play in the life cycle?",
    "What does the 'Industrial Revolution' mean in historical terms?",
    "What is the relationship between capitalism and the free market?",
    "What does 'blockchain' mean in the technological context?",
    "What is the importance of photosynthesis in nature?",
    "What role does education play in economic development?"
]



# Iterar sobre las preguntas y obtener respuestas
for i, question in enumerate(questions, 1):
    print(f"\nPregunta {i}: {question}")
    
    # Llamar al método para generar la respuesta
    response = answer_question_word2vec(
        df=df,
        question=question,
        word2vec_model=model,
        idf_dict=idf_dict,
        top_n=20,  # Obtener los contextos más relevantes
        gpt_model="gpt-3.5-turbo",
        max_tokens=150,
        debug=False  # Desactiva el modo de depuración para respuestas limpias
    )
    
    # Imprimir la respuesta generada
    print(f"Respuesta: {response}")



Pregunta 1: What does 'Python' mean in programming?
Respuesta: Python is a high-level, general-purpose programming language.

Pregunta 2: What does 'Python' mean in zoology?
Respuesta: En zoología, "Python" se refiere a una serpiente constrictora no venenosa que pertenece a la familia Pythonidae.

Pregunta 3: What is Java in computer science?
Respuesta: Java in computer science is a programming language that allows for the development of software applications that can run on any computer with a Java Virtual Machine installed. This feature enables Java programs to be platform-independent, meaning they can run on different types of computers without needing to be rewritten for each specific system.

Pregunta 4: What is Java in geography?
Respuesta: Java in geography refers to an island of Indonesia.

Pregunta 5: What is the connection between the Sun and photosynthesis?
Respuesta: La conexión entre el Sol y la fotosíntesis es que las células fotosintéticas utilizan la energía del Sol pa

# PREGUNTAS ANALOGICAS

In [24]:
from scipy.spatial.distance import cosine

def find_analogy(word_a, word_b, word_c, model, top_n=1):
    """
    Encuentra las palabras que completan la analogía: word_a es a word_b como word_c es a ?
    
    Parámetros:
        word_a (str): Primera palabra (e.g., "king").
        word_b (str): Segunda palabra (e.g., "queen").
        word_c (str): Tercera palabra (e.g., "man").
        model: Modelo Word2Vec ya entrenado.
        top_n (int): Número de palabras más cercanas a retornar.
    
    Retorna:
        list: Lista de las palabras más cercanas junto con su similitud.
    """
    try:
        # Convertir las palabras a minúsculas
        word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
        
        # Verificar si las palabras están en el vocabulario
        for word in [word_a, word_b, word_c]:
            if word not in model.wv.key_to_index:
                return [f"'{word}' no está en el vocabulario del modelo."]
        
        # Calcular el vector resultante para la analogía
        analogy_vector = model.wv[word_a] - model.wv[word_b] + model.wv[word_c]
        
        # Encontrar las palabras más similares al vector resultante
        similar_words = model.wv.similar_by_vector(analogy_vector, topn=top_n + 3)  # Extraemos más palabras para filtrar
        # Excluir las palabras originales de la analogía
        filtered_words = [(word, similarity) for word, similarity in similar_words if word not in {word_a, word_b, word_c}]
        
        return filtered_words[:top_n]

    except KeyError as e:
        return [f"Error: {e}"]

    except Exception as e:
        return [f"Error inesperado: {e}"]


In [25]:
# Pregunta 1
spanish_question = "Rey es a reina como hombre es a ¿qué?"

word_a, word_b, word_c = "king", "queen", "man" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

king es a queen como man es a: [('soldier', 0.5685295462608337), ('warrior', 0.5266982316970825), ('vortigern', 0.5246458649635315)]


In [26]:
# Pregunta 2
spanish_question = "Madrid es a España como Tokio es a ¿qué?"

word_a, word_b, word_c = "madrid", "spain", "tokio" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

madrid es a spain como tokio es a: [('chelsea', 0.5345818400382996), ('fulham', 0.5259823203086853), ('luton', 0.48761096596717834)]


In [27]:
# Pregunta 3
spanish_question = "Día es a sol como noche es a ¿qué?"

word_a, word_b, word_c = "day", "sunny", "night" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

day es a sunny como night es a: [('week', 0.8130009770393372), ('hour', 0.76027512550354), ('morning', 0.7474613785743713)]


In [28]:
# Pregunta 4
spanish_question = "La cuchara es a sopa como tenedor es a ¿qué?"

word_a, word_b, word_c = "spoon", "soup", "fork" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

spoon es a soup como fork es a: [('block', 0.5301869511604309), ('fishermans', 0.5300419926643372), ('stack', 0.524989128112793)]


In [29]:
# Pregunta 5
spanish_question = "Manzana es a fruta como perro es a ¿qué?"

word_a, word_b, word_c = "apple", "fruit", "dog" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

apple es a fruit como dog es a: [('kermit', 0.5665644407272339), ('trs-80', 0.5645706653594971), ('capcom', 0.5327814221382141)]


In [30]:
# Pregunta 6
spanish_question = "Ojo es a ver cómo oreja es a ¿qué?"

word_a, word_b, word_c = "eye", "see", "ear" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

eye es a see como ear es a: [('eyelid', 0.7427768111228943), ('fingertip', 0.7015135288238525), ('nostril', 0.6950225234031677)]


In [31]:
# Pregunta 7
spanish_question = "El carro es a combustible como humano es a ¿qué?"

word_a, word_b, word_c = "car", "fuel", "human" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

car es a fuel como human es a: [('person', 0.5392000079154968), ('individual', 0.47591862082481384), ('universal', 0.4596824049949646)]


In [32]:
# Pregunta 8
spanish_question = "Ave es a volar como pez es a ¿qué?"

word_a, word_b, word_c = "bird", "flying", "fish" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

bird es a flying como fish es a: [('salmon', 0.7940330505371094), ('seabird', 0.7700880765914917), ('crocodile', 0.7623776197433472)]


In [33]:
# Pregunta 9
spanish_question = "Computadora es a procesador como humano es a ¿qué?"

word_a, word_b, word_c = "computer", "processor", "human" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

computer es a processor como human es a: [('animal', 0.5930473804473877), ('sentient', 0.5728640556335449), ('humanity', 0.5690764784812927)]


In [34]:
# Pregunta 10
spanish_question = "Libro es a leer como película es a ¿qué?"

word_a, word_b, word_c = "book", "read", "movie" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

book es a read como movie es a: [('film', 0.7438424229621887), ('sequel', 0.684289813041687), ('trilogy', 0.6735404133796692)]
